In [6]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from playwright.async_api import async_playwright
import pandas as pd

# List of license numbers to scrape
license_numbers = ["006565540C", "0654479VSF", "006564940C"]

async def scrape_license(license_number, headless=False):
    async with async_playwright() as p:
        browser = await p.firefox.launch(headless=headless)
        page = await browser.new_page()
        await page.goto("https://www.tdlr.texas.gov/tools_search/", timeout=60000)

        # Click TDLR Number radio button
        await page.click('#mcrbutton')

        # Fill license number
        await page.fill('#mcrdata', license_number.upper())

        # Click Search
        await page.click('#submit3')

        # Wait a bit for results
        await page.wait_for_timeout(3000)

        # Check for "No results"
        no_results = await page.query_selector("text=No results")
        if no_results:
            await browser.close()
            return {
                "License": license_number,
                "Company Name": "No results",
                "DBA Name": "",
                "Owners": "",
                "Phone": "",
                "Certificate Number": "",
                "Carrier Type": "",
                "Mailing Address": "",
                "Physical Address": "",
                "Violations": ""
            }

        # Extract company info from tables
        tables = await page.query_selector_all("table")
        company_info = {
            "License": license_number,
            "Company Name": "",
            "DBA Name": "",
            "Owners": [],
            "Phone": "",
            "Certificate Number": "",
            "Carrier Type": "",
            "Mailing Address": "",
            "Physical Address": "",
            "Violations": ""
        }

        for table in tables:
            tds = await table.query_selector_all("td")
            for td in tds:
                text = (await td.inner_text()).strip()
                if text.startswith("Name:"):
                    company_info["Company Name"] = text.replace("Name:", "").strip()
                elif text.startswith("DBA:"):
                    company_info["DBA Name"] = text.replace("DBA:", "").strip()
                elif text.startswith("Owner/Officer:"):
                    company_info["Owners"].append(text.replace("Owner/Officer:", "").strip())
                elif text.startswith("Phone:"):
                    company_info["Phone"] = text.replace("Phone:", "").strip()
                elif text.startswith("Number:"):
                    company_info["Certificate Number"] = text.replace("Number:", "").split("(")[0].strip()
                elif text.startswith("Carrier Type:"):
                    # Carrier type and addresses are together
                    lines = text.split("\n")
                    company_info["Carrier Type"] = lines[0].replace("Carrier Type:", "").strip()
                    # Mailing and Physical addresses
                    for i, line in enumerate(lines):
                        if line.startswith("Mailing:"):
                            addr_lines = []
                            for l in lines[i+1:]:
                                if l.startswith("Physical:") or l.strip() == "":
                                    break
                                addr_lines.append(l.strip())
                            company_info["Mailing Address"] = " ".join(addr_lines)
                        if line.startswith("Physical:"):
                            addr_lines = []
                            for l in lines[i+1:]:
                                if l.strip() == "":
                                    break
                                addr_lines.append(l.strip())
                            company_info["Physical Address"] = " ".join(addr_lines)

        # Convert owners list to string
        company_info["Owners"] = "; ".join(company_info["Owners"]) if company_info["Owners"] else ""

        # Extract violations (if any)
        violations_table = await page.query_selector('table[bordercolor="#990000"]')
        if violations_table:
            v_rows = await violations_table.query_selector_all("tr")
            violations = []
            for v_row in v_rows[1:]:  # skip header
                v_cells = await v_row.query_selector_all("td")
                v_text = [await cell.inner_text() for cell in v_cells]
                violations.append(" | ".join([t.strip() for t in v_text]))
            company_info["Violations"] = "\n".join(violations) if violations else "None"
        else:
            company_info["Violations"] = "None"

        await browser.close()
        return company_info

async def scrape_all_licenses():
    results = []
    for lic in license_numbers:
        print(f"Scraping {lic}")
        data = await scrape_license(lic, headless=False)  # headless=False to see Firefox
        results.append(data)
    df = pd.DataFrame(results)
    return df

# Run scraper
loop = asyncio.get_event_loop()
license_data = loop.run_until_complete(scrape_all_licenses())

# Save to CSV
license_data.to_csv("license_info_violations.csv", index=False)
print(license_data)





Scraping 006565540C
Scraping 0654479VSF
Scraping 006564940C
      License              Company Name                     DBA Name  \
0  006565540C                                                          
1  0654479VSF  24/7TOWINGANDRECOVERYLLC  MIDLAND TOWING AND RECOVERY   
2  006564940C              A&NTOWINGLLC                      NO DATA   

                                              Owners       Phone  \
0                                                                  
1  SCOTT JENSON / VICE PRESIDENT; CHRIS BRENES / ...  4325576733   
2  ABDULRAHMAN Z ALABDULLAH / CEO; ABDULRAHMAN Z ...  2106678546   

  Certificate Number              Carrier Type  \
0                                                
1         0654479VSF  Vehicle Storage Facility   
2         006564940C         Tow Truck Company   

                              Mailing Address  \
0                                               
1  3601 N COUNTY ROAD 1148 MIDLAND, TX. 79705   
2  10838 DEEPWATER BAY SAN ANT